# 思路：先对两张图像分别分类后，再用分类后得到的分类图像做差，如果结果图像中的值为0，说明没有发生变化，若不为0，则说明发生了变化。

该论文提供了数据集：https://www.mdpi.com/2072-4292/16/13/2355

使用的数据集：https://ieee-dataport.org/open-access/hrscd-high-resolution-semantic-change-detection-dataset
注：labels_land_cover_2006和labels_land_cover_2012文件夹中的文件虽然文件名相同，但是图像的像素值不相同。
地区： The images cover a range of urban and countryside areas around the French cities of Rennes and Caen

In [ ]:
# CNN需要原图 + 已分类的图，CNN自动提取特征，不需要自己提取特征
# 监督式学习分类：你需提供特征, label
# 非监督式学习分类：你需提供特征

# 训练模型

# 评估模型

# 对未见过的图像进行分类

# 比较

In [ ]:
import numpy as np
from osgeo import gdal
# 假装下面两张图片是模型的预测结果
ds_2012 = gdal.Open(r'D:\Download\HRSCD\labels_land_cover_2012\2012\D14\14-2012-0435-6910-LA93-0M50-E080.tif') 
ds_2006 = gdal.Open(r'D:\Download\HRSCD\labels_land_cover_2006\2006\D14\14-2012-0435-6910-LA93-0M50-E080.tif')
band_2012 = ds_2012.GetRasterBand(1)
land_cover_2012 = band_2012.ReadAsArray()

band_2006 = ds_2006.GetRasterBand(1)
land_cover_2006 = band_2006.ReadAsArray()

diff = land_cover_2012.astype(np.int8) - land_cover_2006.astype(np.int8)
print(np.unique(diff))
del ds_2012, ds_2006

[-2 -1  0  1]


## 量化land cover变化情况

In [ ]:
from sklearn.metrics import confusion_matrix
y_true = land_cover_2006.ravel()
y_pred = land_cover_2012.ravel()

# 混淆矩阵可表示land cover的变化情况，
# 例如：以下混淆矩阵第一行表示有115444个像素由2006年的Artificial surfaces变成了2012年的Agricultural areas
labels = np.unique(np.concatenate([y_true, y_pred]))
cm = confusion_matrix(y_true, y_pred, labels=labels)

print("Labels:", labels)
print("Change Matrix:\n", )
print(cm)


Labels: [1 2 3 5]
Change Matrix:

[[ 9277926   115444        0        0]
 [  379229 88554646        0        0]
 [   15631        0  1616793        0]
 [       0        0        0    40331]]
